# <font color='red'>STOCK MARKET IS VERY RISKY UNTIL YOU DO IT PROPERLY. PLEASE DO NOT TAKE TRADES JUST BECAUSE THIS TOOL GIVES YOU THE NAME. APPLY YOUR OWN LEARNINGS, CREATE YOUR OWN STRATEGY, ASSESS RISK & TRADE THE PLAN.</font>

In [1]:
'065676'

'065676'

# Imports

In [2]:
# import backtrader
from helpers.datahandler import *
from helpers.investing import *
from helpers.journal_handler import *
from helpers.intraday import *
from helpers.nse_data import *
from helpers.backtest import *

# Defaults

In [3]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 25)

In = Investing(check_fresh = True) # Investing or Swing trading analysis
Intra = IntraDay() # Intraday strategies and analysis
NSE = NSEData() # Get NSE Live data
MS = MarketSentiment() # Get Live Market Sentiment TICK, TRIN and 52W high/ low stocks
BT = Backtest()

Checking Fresh Data.....


# Journal
Make a journal in Google Drive for each trade you take to analyse. It is a not so simple process and need to connect with Google API so if you're not comfortable with it, just skip it

In [4]:
private_key = join(expanduser('~'),'Documents','client_secret.json') # Personal for each user. Mine is stored in /home/Documents
JH = JournalHandler(private_key)

journal = JH.get_journal('Finance Journal') # My Google Sheet jornam has name "Finance Journal"

JH.total_pl(journal) # total Profit and Loss up until now

These stocks have crossed 21 days limit. Sell them at 1:1.5 or 1:1 or at Market Price
ICICIBANK bought on 2021-09-01 00:00:00

--------------------------------------------------------------------------- 

Trailing Stop Loss: 

CESC bought on: 2021-09-28 00:00:00,

--------------------------------------------------------------------------- 



/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


524.11

# <font color='teal'>Swing Strategies </font>
## All Details
Basic Details about stocks such as Momentum, Ichimoku, 1-2-3 candles pattern etc

In [5]:
budget = 40000

df = In.get_recent_info(nifty = 100, **{'mvs':[20,50,100,200]})
df.head(2)

/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


,Date,Name,LTP,Index,CCI Signal,RSI Signal,MACD Signal,Over 20-SMA,Over 50-SMA,Over 100-SMA,Over 200-SMA,Momentum ADX,Ichi Count,1 Candle,2 Candles,3 Candles
0,2021-10-05,ACC,2263.7,Nifty 100,Buy,No Signal,No Signal,False,False,True,True,25.212109,0,Unknown Green,Unknown,Unknown
1,2021-10-05,ABBOTINDIA,23184.5,Nifty 100,No Signal,No Signal,No Signal,True,True,True,True,33.095309,2,Shooting Star,Unknown,Unknown


## 44-SMA 
If you find a `Green` candle taking support on 44 days `Simple Moving Average` line and stock is in uptrend, you can take the trade and set Risk to Reward ratio as 1:2

In [6]:
budget = 40000 # Total Budget
risk = 400 # Risk per trade
df = In.show_full_stats(budget=budget,risk=risk, diff = 100, nifty='nifty_100') # Show only stocks undder your budget and  whose recent candle's lowest is not more than 151 rupees above the moving average line
df[(df['RSI Value'] < 70) & (df['CCI Value'] < 200)]

,DATE,OPEN,HIGH,LOW,CLOSE,52W H,52W L,SYMBOL,Diff,CCI Value,RSI Value,MACD Signal,Direction,Ichi,ATR,Triple Candle,Double Candle,Recent Candle,Index,Expected Change %,Max Config Risk
14,2021-10-05,2730.00,2789.00,2729.05,2760.10,3037.00,2015.20,EICHERMOT,0.42,-129.01,46.918566,No Signal,Undeterministic,0,72.210714,V Pattern,Bullish Engulfing,Unknown Green,Nifty 50,6.19,345.99
12,2021-10-05,774.40,784.00,771.20,782.60,820.70,439.05,AXISBANK,0.45,-91.34,49.905260,No Signal,Rising High,0,18.925000,Unknown,Unknown,Unknown Green,Nifty 50,3.89,396.61
26,2021-10-04,639.90,643.60,635.95,641.05,699.15,315.20,WIPRO,0.80,-122.60,45.949968,No Signal,Undeterministic,0,14.907143,V Pattern,Unknown,Green Doji,Nifty 50,4.55,396.19
20,2021-10-05,819.40,828.00,815.60,825.45,872.95,584.30,BERGEPAINT,0.82,-18.67,52.076021,No Signal,Undeterministic,1,15.932143,Unknown,Unknown,Unknown Green,Nifty 100,4.50,391.42
2,2021-10-05,1542.25,1588.00,1541.55,1548.70,1675.00,1200.00,ICICIGI,1.45,-144.03,43.900323,No Signal,Undeterministic,0,45.342857,Unknown,Unknown,Inverted Hammer,Nifty 100,6.23,396.64
5,2021-10-05,299.70,308.00,299.20,303.00,341.45,91.20,VEDL,1.64,55.87,54.393987,No Signal,Undeterministic,0,9.575000,Unknown,Unknown,Unknown Green,Nifty 100,14.55,381.55
31,2021-10-05,562.00,567.05,555.60,563.95,673.70,395.80,CADILAHC,1.74,66.42,53.645621,No Signal,Undeterministic,1,14.200000,Reverse V Pattern,Unknown,Green Doji,Nifty 100,4.43,389.75
16,2021-10-05,672.50,685.50,667.95,678.90,724.30,398.20,ICICIPRULI,2.39,-50.14,51.269224,No Signal,Undeterministic,0,18.739286,Unknown,Unknown,Unknown Green,Nifty 100,7.70,396.69
3,2021-10-05,1399.10,1421.00,1381.00,1413.05,1547.85,781.85,TECHM,4.93,-79.22,49.178093,No Signal,Undeterministic,0,42.521429,Unknown,Unknown,Unknown Green,Nifty 50,7.27,362.20
0,2021-10-05,295.10,299.50,292.75,298.40,430.70,259.25,BANDHANBNK,5.52,189.91,60.521373,No Signal,Undeterministic,1,8.810714,Unknown,Unknown,Unknown Green,Nifty 100,6.06,399.98


## Golden Crossover
When 44 days `SMA` Crosses 200 days `SMA` from below and you find a `Green` candle within 15 days taking support on `44` days line set Risk to Reward ratio as 1:5

In [7]:
for name in In.data['nifty_200']:
    df = In.open_downloaded_stock(name)
    if In.has_golden_crossover(df,lookback=21):
        print(name)

ESCORTS
HDFCLIFE
ITC
INDIAMART
KOTAKBANK
ZEEL


## RSI Signal: Oversold / Overbought Stocks
Buy when RSI is less than 30 sell when greater than 70

In [8]:
for name in In.data['nifty_200']:
    df = In.open_downloaded_stock(name)
    if In.get_RSI(df) < 30: # Oversold
        print(name)

CESC


## MACD Signal
It is the strategy when MACD (Blue) Line cuts the Signal (Red) line from below then it is a buy signal. When the blue line cuts from upside down, it is sell signal

In [9]:
for name in In.data['nifty_200']:
    df = In.open_downloaded_stock(name)
    if In.macd_signal(df) == 'Buy':
        print(name)

ADANIENT
ADANIGREEN
BPCL
DEEPAKNTR
FORTIS
GLAND
IDFCFIRSTB
IRCTC
LAURUSLABS
MUTHOOTFIN
PAGEIND
SBICARD
SBILIFE
SANOFI
SIEMENS


## CCI Signal : Rally
**Faster than MACD Signal works closely with Stochastic RSI**

When CCI values comes from below  -100 to above -100, buy signal and when it comes from above +100 to below +100, sell signal

In [10]:
for name in In.data['nifty_100']:
    df = In.open_downloaded_stock(name)
    if In.get_CCI(df, signal_only = True) == 'Buy':
        print((name, In.get_index(name)))

('ACC', 'Nifty 100')
('AXISBANK', 'Nifty 50')
('DIVISLAB', 'Nifty 50')
('GODREJCP', 'Nifty 100')
('HINDUNILVR', 'Nifty 50')
('INFY', 'Nifty 50')
('SHREECEM', 'Nifty 50')
('TCS', 'Nifty 50')
('TECHM', 'Nifty 50')
('UPL', 'Nifty 50')
('ULTRACEMCO', 'Nifty 50')


# <font color='teal'>Intraday Strategies</font>

## Narrow Range 7
If recent candle's range is lowest among 6 other days then the future candle might break it's low/high

In [11]:
for name in In.data['nifty_100']:
    if Intra.NR7_strategy(name):
        print(name)

AMBUJACEM
DMART
BAJAJ-AUTO
BIOCON
BOSCHLTD
COLPAL
DLF
GODREJCP
GRASIM
HDFCAMC
HDFCBANK
HAVELLS
HEROMOTOCO
ITC
INDUSTOWER
INDUSINDBK
INDIGO
JSWSTEEL
JUBLFOOD
KOTAKBANK
LUPIN
MRF
M&M
MARICO
MUTHOOTFIN
NESTLEIND
PIDILITIND
SHREECEM
SIEMENS
TORNTPHARM
ULTRACEMCO
WIPRO


## Whole Number Open
When Open == Low / High in Whole number as `xxx.00` within 15 minutes

In [12]:
result = Intra.whole_number_strategy(nifty=200, min_val=101,max_val=4999)
for key in result.keys():
    print(key,":\n")
    for r in sorted(result[key],key=lambda x: int(x[-1].split(' ')[-1])):
        print(r,'\n')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## `%` Change probability
Gives the stock names which have the highest probability of providing you `x%` in intraday atleast on both long or short side based on the previous `N` days record

In [ ]:
# Gives you top 10 stocks from Nifty 500 which have the highest probability of giving atleast 0.99% on your investment based on the Long/Buy position. Data is based on past 60 trading days
# Test it for different time duration, index, % change and Long / Short criteria based on overall market trend

Intra.prob_by_percent_change(symbol = None, index = 500, time_period = 60, change_percent=0.099, sort_by='Long Probability', top_k = 10) 

# <font color='Teal'>Live Data</font>


## Market Sentiment: TICK, TRIN (Arm's Index) & 52W high-Low touching shares


In [13]:
MS.get_live_sentiment()

{'Latest Updated on': '05/10/2021 16:14:59',
 'Up': 923,
 'Down': 579,
 'TICK': 344,
 'Volume Up': 2262.86,
 'Volume Down': 830.76,
 'TRIN': 0.585,
 '52W High': 122,
 '52W Low': 2}

## Indices Based
Buy or Sell stock based on which indices are performing highest in first 5 mins. Buy or sell stocks only for those indices which arevperforming the greatest

In [ ]:
NSE.current_indices_status(10) # Live as of market is open. works post market also

In [ ]:
NSE.open_nse_index('NIFTY PSE')

# <font color='teal'>Back Testing</font>
# Back Tersting Stratagies to get the best Win %, Max Return Investment etc
Pass in the parameters dict for all. Check Docstring of each individual function for how a strategy works in buying or selling.

## CCI
`BT.backtest('cci', stocks=['DIVISLAB'])` , for individual stock data performance to tell how much a stock would have gained or lost

In [14]:
cci_parameters = {'buying_thresh':-100, 'selling_thresh':100, 'window':20, 'cols':('OPEN','CLOSE','LOW','HIGH', 'DATE')} # check documentation of BT.cci()

# Using the CCI strategy, Return top gaining stocks which have traded atleast 365 recent days and are currently in nifty-500
BT.backtest('cci', min_days = 365, stocks = 'nifty_50', **cci_parameters)

,win%,wins,losses,ROI,p&l,buy_date,sell_date,buy_price,sell_price,hold_period,buys,sells,days
TITAN,1.0,7,0,3.22,"[16.0, 77.90000000000009, 54.399999999999864, ...","[2020-03-27 00:00:00, 2020-09-29 00:00:00, 202...","[2020-06-11 00:00:00, 2020-10-15 00:00:00, 202...","[960.0, 1145.0, 1224.15, 1518.0, 1460.0, 1505....","[976.0, 1222.9, 1278.55, 1536.7, 1485.6, 1536....","[76, 16, 14, 24, 21, 27, 20]",7,7,406
UPL,1.0,6,0,9.36,"[17.399999999999977, 15.800000000000011, 37.39...","[2020-03-27 00:00:00, 2020-07-20 00:00:00, 202...","[2020-04-22 00:00:00, 2020-07-27 00:00:00, 202...","[327.6, 440.0, 492.6, 447.4, 611.0, 809.0, 778.0]","[345.0, 455.8, 530.0, 456.0, 811.7, 821.75]","[26, 7, 12, 43, 46, 20]",7,6,408
BAJAJFINSV,1.0,6,0,9.22,"[190.0, 475.0, 120.05000000000018, 2631.100000...","[2020-03-30 00:00:00, 2020-05-21 00:00:00, 202...","[2020-05-05 00:00:00, 2020-06-11 00:00:00, 202...","[4700.0, 4725.0, 6279.95, 6118.9, 8900.05, 954...","[4890.0, 5200.0, 6400.0, 8750.0, 9144.0, 9600.0]","[36, 21, 6, 76, 20, 21]",6,6,407
DIVISLAB,1.0,5,0,13.94,"[445.04999999999995, 946.6500000000001, 362.0,...","[2020-03-23 00:00:00, 2020-07-13 00:00:00, 202...","[2020-04-21 00:00:00, 2020-08-19 00:00:00, 202...","[1849.95, 2220.0, 3080.0, 3590.0, 4863.45]","[2295.0, 3166.65, 3442.0, 3790.0, 5084.2]","[29, 37, 46, 72, 15]",5,5,407
TATASTEEL,1.0,5,0,10.32,"[12.050000000000011, 28.399999999999977, 99.05...","[2020-03-23 00:00:00, 2020-09-30 00:00:00, 202...","[2020-04-21 00:00:00, 2020-10-29 00:00:00, 202...","[268.0, 371.05, 644.9, 730.4, 1114.2, 1399.0]","[280.05, 399.45, 743.95, 862.0, 1166.0]","[29, 29, 33, 18, 10]",6,5,407
INFY,1.0,5,0,4.99,"[136.89999999999998, 15.799999999999955, 10.04...","[2020-03-23 00:00:00, 2020-08-27 00:00:00, 202...","[2020-05-06 00:00:00, 2020-09-25 00:00:00, 202...","[540.0, 960.0, 1109.9, 1277.65, 1342.0]","[676.9, 975.8, 1119.95, 1330.0, 1388.1]","[44, 29, 21, 44, 15]",5,5,407
GRASIM,1.0,4,0,9.40,"[170.0, 15.0, 82.54999999999995, 97.2999999999...","[2020-03-30 00:00:00, 2020-07-21 00:00:00, 202...","[2020-06-11 00:00:00, 2020-08-12 00:00:00, 202...","[448.0, 605.0, 1337.45, 1489.95]","[618.0, 620.0, 1420.0, 1587.25]","[73, 22, 16, 28]",4,4,408
WIPRO,1.0,4,0,5.51,"[39.599999999999994, 5.5, 0.9499999999999886, ...","[2020-03-25 00:00:00, 2020-09-01 00:00:00, 202...","[2020-06-11 00:00:00, 2020-09-07 00:00:00, 202...","[177.4, 271.0, 417.95, 561.0]","[217.0, 276.5, 418.9, 593.6]","[78, 6, 48, 14]",4,4,407
CIPLA,0.8,9,1,6.43,"[166.2, 43.0, 26.950000000000045, 21.75, 8.599...","[2020-03-18 00:00:00, 2020-05-19 00:00:00, 202...","[2020-04-21 00:00:00, 2020-06-04 00:00:00, 202...","[411.8, 600.0, 637.0, 742.7, 761.2, 749.5, 839...","[578.0, 643.0, 663.95, 764.45, 769.8, 764.0, 8...","[34, 16, 24, 21, 15, 27, 9, 59, 16, 30]",10,10,408
ITC,0.8,9,1,4.67,"[12.900000000000006, 33.25, 14.599999999999994...","[2020-03-20 00:00:00, 2020-05-14 00:00:00, 202...","[2020-04-21 00:00:00, 2020-06-04 00:00:00, 202...","[165.1, 164.5, 185.4, 197.65, 174.75, 207.45, ...","[178.0, 197.75, 200.0, 198.05, 182.8, 214.0, 2...","[32, 21, 14, 25, 51, 9, 27, 35, 42, 14]",10,10,407


## RSI

In [15]:
BT.backtest('rsi',stocks=['BAJFINANCE','SHREECEM',"COALINDIA"]) # Individual stocks. If no sells have been made, it'll be empty

,win%,wins,losses,ROI,p&l,buy_date,sell_date,buy_price,sell_price,hold_period,buys,sells,days
SHREECEM,1.00,2,0,12.64,"[1749.5, 4290.0999999999985]","[2020-03-11 00:00:00, 2021-08-17 00:00:00]","[2020-11-13 00:00:00, 2021-09-03 00:00:00]","[21780.0, 26000.0]","[23529.5, 30290.1]","[247, 17]",2,2,407
COALINDIA,0.50,3,1,7.33,"[-15.400000000000006, 12.199999999999989, 17.3...","[2020-03-11 00:00:00, 2020-09-22 00:00:00, 202...","[2020-08-27 00:00:00, 2020-12-04 00:00:00, 202...","[158.1, 122.0, 129.1, 136.3]","[142.7, 134.2, 146.45, 162.15]","[169, 73, 46, 24]",4,4,408
BAJFINANCE,0.33,2,1,17.24,"[-1000.0, 818.9499999999998, 2204.8999999999996]","[2020-03-06 00:00:00, 2020-09-25 00:00:00, 202...","[2020-06-24 00:00:00, 2020-11-10 00:00:00, 202...","[4125.0, 3080.0, 4535.1]","[3125.0, 3898.95, 6740.0]","[110, 46, 132]",3,3,408


## MACD

In [16]:
BT.backtest('macd', top_n=5, stocks='nifty_500')

,win%,wins,losses,ROI,p&l,buy_date,sell_date,buy_price,sell_price,hold_period,buys,sells,days
APOLLOHOSP,0.82,10,1,7.04,"[54.399999999999864, 27.799999999999955, 277.4...","[2020-04-08 00:00:00, 2020-05-27 00:00:00, 202...","[2020-05-08 00:00:00, 2020-07-01 00:00:00, 202...","[1225.9, 1322.2, 1427.0, 1777.1, 2275.0, 2522....","[1280.3, 1350.0, 1704.4, 2224.3, 2375.8, 2590....","[30, 35, 41, 31, 30, 26, 27, 36, 13, 37, 21]",11,11,408
DALBHARAT,0.75,7,1,10.09,"[33.0, 306.54999999999995, 15.0, 149.349999999...","[2020-08-04 00:00:00, 2020-09-17 00:00:00, 202...","[2020-08-17 00:00:00, 2020-12-10 00:00:00, 202...","[754.0, 771.45, 1115.0, 1266.65, 1590.9, 1560....","[787.0, 1078.0, 1130.0, 1416.0, 1582.0, 1778.4...","[13, 84, 14, 23, 3, 27, 28, 27]",8,8,408
HAL,0.60,8,2,10.02,"[358.29999999999995, 9.099999999999909, -87.0,...","[2020-05-15 00:00:00, 2020-08-11 00:00:00, 202...","[2020-07-16 00:00:00, 2020-08-27 00:00:00, 202...","[510.0, 1032.0, 813.0, 697.9, 703.8, 879.8, 10...","[868.3, 1041.1, 726.0, 680.0, 860.1, 946.0, 10...","[62, 16, 20, 5, 45, 22, 28, 43, 15, 29]",10,10,408
SKFINDIA,0.56,7,2,3.36,"[72.5, 157.20000000000005, 62.950000000000045,...","[2020-08-14 00:00:00, 2020-10-09 00:00:00, 202...","[2020-09-09 00:00:00, 2020-12-22 00:00:00, 202...","[1508.0, 1477.8, 1737.35, 2025.0, 2222.1, 2535...","[1580.5, 1635.0, 1800.3, 2283.5, 2342.15, 2614...","[26, 74, 21, 19, 35, 18, 6, 17, 20]",9,9,408
VAIBHAVGBL,0.50,9,3,6.02,"[189.79999999999995, 111.09999999999991, -14.9...","[2020-05-26 00:00:00, 2020-08-05 00:00:00, 202...","[2020-06-29 00:00:00, 2020-08-21 00:00:00, 202...","[1067.95, 1612.9, 1815.05, 1880.0, 2000.0, 226...","[1257.75, 1724.0, 1800.1, 1883.0, 2139.0, 2614...","[34, 16, 5, 7, 28, 20, 4, 28, 5, 3, 85, 22]",12,12,408


## MA
Moving Average Strategy

In [ ]:
BT.backtest('ma',stocks='all',top_n=25)

# <font color='teal'>Risk Management</font>

In [ ]:
# You have a total budget of 40K, and a risk apetite of rs 400 per trade
# You can set manual such as # entry = 3980.1, stop_loss=3780.5

In.get_particulars('NIITLTD',40000,400,entry = 368.5, stop_loss=319)